In [2]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from yahoofinancials import YahooFinancials

# Correct the file path and quote characters
ticker_details = pd.read_excel(r'Data\Ticker_List.xlsx')

# Retrieve tickers and names
ticker = ticker_details['Ticker'].to_list()
names = ticker_details['Description'].to_list()

# Display the first 20 rows
ticker_details.head(20)


,Ticker,Description
0,GC=F,Gold
1,SI=F,Silver
2,CL=F,Crude Oil
3,^GSPC,S&P500
4,^RUT,Russel 2000 Index
5,ZN=F,10 Yr US T-Note futures
6,ZT=F,2 Yr US T-Note Futures
7,PL=F,Platinum
8,HG=F,Copper
9,DX=F,Dollar Index


In [3]:
# Creating Date Range and adding them to values table
end_date = "2020-03-01"
start_date = "2010-01-01"
date_range = pd.bdate_range(start=start_date, end=end_date)
values = pd.DataFrame({'Date': date_range})
values['Date'] = pd.to_datetime(values['Date'])


In [4]:


# Define the date range
end_date = "2020-03-01"
start_date = "2010-01-01"
date_range = pd.bdate_range(start=start_date, end=end_date)

# Create the values DataFrame with dates
values = pd.DataFrame({'Date': date_range})
values['Date'] = pd.to_datetime(values['Date'])

# List of ticker codes
# Make sure `ticker` and `names` are defined lists of tickers and descriptions respectively

for i in ticker:
    raw_data = YahooFinancials(i)
    data = raw_data.get_historical_price_data(start_date, end_date, "daily")
    
    # Check if there is data available for the ticker
    if i in data:
        df = pd.DataFrame(data[i]['prices'])[['formatted_date', 'adjclose']]
        df.columns = ['Date1', i]  # Rename columns to 'Date1' and the ticker symbol
        df['Date1'] = pd.to_datetime(df['Date1'])
        
        # Merge with the values DataFrame on the Date column
        values = values.merge(df, how='left', left_on='Date', right_on='Date1')
        values = values.drop(labels='Date1', axis=1)  # Drop the auxiliary 'Date1' column

# Rename columns to more descriptive names based on 'names' list
names.insert(0, 'Date')
values.columns = names

print(values.shape)
print(values.isna().sum())

# Fill NaN values using forward and backward fill
values = values.fillna(method="ffill", axis=0)
values = values.fillna(method="bfill", axis=0)
print(values.isna().sum())

# Convert all columns except 'Date' to numeric, coercing errors to NaN, and round to 1 decimal place
cols = values.columns.drop('Date')
values[cols] = values[cols].apply(pd.to_numeric, errors='coerce').round(decimals=1)
print(values.tail())


KeyboardInterrupt: 

In [ ]:

imp = ['Gold', 'Silver', 'Crude Oil', 'S&P500', 'MSCI EM ETF']

# Calculating Short-term Historical Returns
change_days = [1, 3, 5, 14, 21]

# Initialize a DataFrame with the 'Date' column
data = pd.DataFrame(data=values['Date'])

for i in change_days:
    print(data.shape)
    # Calculate percentage change and add suffix indicating the time period
    x = values[cols].pct_change(periods=i).add_suffix("-T-" + str(i))
    data = pd.concat([data, x], axis=1)

print(data.shape)

# Calculating Long-term Historical Returns
change_days = [60, 90, 180, 250]

for i in change_days:
    print(data.shape)
    # Calculate percentage change for the selected important columns
    x = values[imp].pct_change(periods=i).add_suffix("-T-" + str(i))
    data = pd.concat([data, x], axis=1)

print(data.shape)
